In [1]:
!pip install python-binance
!pip install matplotlib --upgrade
!pip install catboost
#!pip install --upgrade mplfinance  
!pip install ta
!pip install sqlalchemy

In [2]:
import numpy as np
import pandas as pd

import time
import dateparser
import pytz
import json
import math  

import datetime as dt
from datetime import datetime, timedelta

from threading import Thread

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.pyplot import figure

#from mpl_finance import candlestick_ochl

#from sklearn import preprocessing,model_selection,neighbors,svm
#from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
#from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

#from catboost import CatBoostClassifier, CatBoostRegressor, Pool, cv

from tqdm import tqdm as tqdm

from ta import add_all_ta_features

from binance.client import Client
from binance import BinanceSocketManager

In [3]:
api_id = ""
api_key = ""
    
client = Client(api_id,api_key)
    
bsm =BinanceSocketManager(client)
    
socket = bsm.trade_socket("BTCUSDT")



In [4]:
print(datetime.now())

2021-08-21 14:00:07.107666


# Indikatörler

In [5]:
def exponential_moving_average(df, n):
  EMA = pd.Series(df['Close'].ewm(span=n, min_periods=n).mean(), name="EMA_" + str(n))
  df = df.join(EMA)
  return df

In [6]:
def moving_average(df,n):
    MA = pd.Series(df["Close"].rolling(n,min_periods=n).mean(),name="MA_"+str(n))
    df = df.join(MA)
    return df

In [7]:
def momentum(df, n):
  M = pd.Series(df['Close'].diff(n), name="Momentum_"+str(n))
  df = df.join(M)
  return df

In [8]:
def bollinger_bands(df, n):
  MA = pd.Series(df['Close'].rolling(n, min_periods=n).mean())
  MSD = pd.Series(df['Close'].rolling(n, min_periods=n).std())
  b1 = 4 * MSD / MA
  B1 = pd.Series(b1, name = "BollingerB_" + str(n))
  df = df.join(B1)
  b2 = (df['Close'] - MA  + 2 * MSD) / (4 * MSD)
  B2 = pd.Series(b2, name='Bollinger%b_'+str(n))
  df = df.join(B2)
  return df

In [9]:
def stochastic_oscillator(df ,n):
  S0K = pd.Series((df['Close']- df['Low']) / (df['High']-df['Low']), name='S0k_' + str(n))
  df = df.join(S0K)
  return df

In [10]:
def relative_strength_index(df, n):
  i = 0
  UpI = [0]
  DoI = [0]

  while i + 1 <= df.index[-1]:
    UpMove = df.loc[i+1, 'High'] - df.loc[i, 'High']
    DoMove = df.loc[i+1, 'Low'] - df.loc[i, 'Low']
    if UpMove > DoMove and UpMove > 0:
      UpD = UpMove
    else:
      UpD = 0
    UpI.append(UpD)
    if DoMove > UpMove and DoMove > 0:
      DoD = DoMove
    else:
      DoD = 0
    DoI.append(DoD)
    i = i + 1

  UpI = pd.Series(UpI)
  DoI = pd.Series(DoI)
  PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean())
  NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean())
  RSI = pd.Series(PosDI / (PosDI + NegDI), name='RSI_' + str(n))
  df = df.join(RSI)
  return df

In [11]:
def common_channel_index(df, n):
  PP = (df['High'] + df['Low'] + df['Close']) / 3
  CCI = pd.Series((PP - PP.rolling(n, min_periods=n).mean()) / PP.rolling(n, min_periods=n).std(), name='CCI_' + str(n))
  df = df.join(CCI)
  return df

In [12]:
def standart_deviation(df, n):
  df = df.join(pd.Series(df['Close'].rolling(n, min_periods=n).std(), name='STD_' + str(n)))
  return df

# Binance Yardımcı Fonksiyonlar


In [13]:
def binanceDataFrame(klines):
  df = pd.DataFrame(klines.reshape(-1,12), dtype=float, columns=['Open Time',
                                                                 'Open',
                                                                 'High',
                                                                 'Low',
                                                                 'Close',
                                                                 'Volume',
                                                                 'Close Time',
                                                                 'Quote asset volume',
                                                                 'Number of trades',
                                                                 'Taker buy base asset volume',
                                                                 'Taker buy quote asset volume',
                                                                 'Can be ignored']);

  df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
  df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')

  return df

In [14]:
def date_to_miliseconds(date_str):
  epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
  d = dateparser.parse(date_str)
  if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
    d = d.replace(tzinfo=pytz.utc)

  #return difference in time
  return int((d - epoch).total_seconds()*1000.0)

In [15]:
def interval_to_miliseconds(interval):
  ms = None
  seconds_per_unit = {
      "m": 60,
      "h": 60 * 60,
      "d": 24 * 60 * 60,
      "w": 7 * 24 * 60 * 60,
  }

  unit = interval[-1]
  if unit in seconds_per_unit:
    try:
      ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
    except:
      pass

  return ms

In [16]:
def get_historical_klines(symbol, interval, start_str, end_str=None):
  output_data=[]
  limit=500
  timeFrame = interval_to_miliseconds(interval)
  start_ts = date_to_miliseconds(start_str)
  end_ts = None
  if end_str:
    end_ts = date_to_miliseconds(end_str)
  
  idx = 0
  symbol_existed = False
  while True:
    #fetch
    temp_data = client.get_klines(symbol = symbol, 
                                  interval = interval, 
                                  limit = limit, 
                                  startTime = start_ts, 
                                  endTime = end_ts)
    
    if not symbol_existed and len(temp_data):
      symbol_existed = True

    if symbol_existed:
      output_data += temp_data

      start_ts = temp_data[len(temp_data) - 1][0] + timeFrame

    idx += 1

    if len(temp_data)<limit:
      break

    if idx % 3 == 0:
      time.sleep(1)

  return output_data

##  Sat-Satın al For Backtest

In [17]:
def satinal(bakiye,harcananpara,price):
  alinanmiktar = harcananpara/price
  kalanbakiye = bakiye - harcananpara
  return kalanbakiye,alinanmiktar

In [18]:
def sat(bakiye,satilanmiktar,price):
  eldeedilenpara = satilanmiktar*price
  mevcutbakiye = bakiye + eldeedilenpara
  return mevcutbakiye

# 2 MA Stratejisi ile al-sat

In [31]:
%%time
#tarih aralıgını buradan ayarlıyoruz
symbol = "BTCUSDT"
start = "14 Apr, 2021"
end = "24 May, 2021"


interval2 = Client.KLINE_INTERVAL_1HOUR
klines2 = get_historical_klines(symbol,interval2,start,end)
klines2 = np.array(klines2)
df = binanceDataFrame(klines2)


remove_cols = [c for c in df.columns if c not in ['Open Time','Close Time','Open','Close','High','Low','Volume']]
df.drop(remove_cols, axis=1, inplace=True)


df = moving_average(df,5)
df = moving_average(df,10)

df.drop(index=df.index[:10], 
        axis=0, 
        inplace=True)

Wall time: 938 ms


In [32]:
## Binance'dan Verileri Alma ve Düzenleme

## Backtest Yapma

In [33]:
bakiye = 10.0
malmiktar = 0.0
maldayiz = False
karsayısı = 0
zararsayısı = 0
trendsayısı = 0

#urunu kac dolarlık alıyosun buradan belirleyebilirsin 
alimmiktari = 10.0
#işleme gırmek ıcın ma5 ma10'un ne kadar ustunde veya altında olmalı burada belırlıyoruz
fark_orani = 10/1000

for i in range(1,df.index.size - 1):
  low1 = df.iloc[i]["Low"]
  high1 = df.iloc[i]["High"]
  open1 = df.iloc[i]["Open"]
  close1 = df.iloc[i]["Close"]
  ma5 = df.iloc[i]["MA_5"]
  ma10 = df.iloc[i]["MA_10"]
  tarih = df.iloc[i]["Close Time"] 

  if(ma5 > ma10 + ma10*fark_orani and maldayiz == False):
        #alım miktarını kumulatif degılde manuel sabıt bır mıktar olsun dersen alt satırdakı parantez ıcınde yazan 2.bakiye yerine alimmiktari yaz
        #suan 1 saatlıkte bir mum ma5 ma10'un ustunde kalacak sekılde kapandıysa satın alım yapıyoruz.tekrar acıklıyorum ma5 ma10'un ustune gectıgı anda degıl ma10'un ustunde bır mum kapattıgı zaman alım yapıyoruz
        bakiye,malmiktar = satinal(bakiye,bakiye,close1)
        maldayiz = True
        print(close1,"dan alım yapıldı mevcut bakiye ",close1*malmiktar,"ma5 = ",ma5,"ma10 = ",ma10," tarih ",tarih)
  elif(ma5 + ma5*fark_orani < ma10 and maldayiz == True):
        #1 saatlikte ma5 ma10'un altında mum kapanısı geldıgı zaman satıs yapıyor
        bakiye = sat(bakiye,malmiktar,close1)
        maldayiz = False
        print(close1,"dan satış yapıldı mevcut bakiye ",bakiye,"ma5 = ",ma5,"ma10 = ",ma10," tarih ",tarih)
if(malmiktar == 0):
    print("en son bakiye ",bakiye)
else:
    print(malmiktar*df.iloc[df.index.size - 1]["Close"])

55336.14 dan alım yapıldı mevcut bakiye  10.0 ma5 =  55926.124000000025 ma10 =  55295.703  tarih  2021-04-20 14:59:59.999000
53787.63 dan satış yapıldı mevcut bakiye  9.720162989323072 ma5 =  54593.153999999995 ma10 =  55187.779999999984  tarih  2021-04-21 23:59:59.999000
50667.75 dan alım yapıldı mevcut bakiye  9.720162989323072 ma5 =  50291.57599999997 ma10 =  49792.186000000016  tarih  2021-04-24 19:59:59.999000
47598.87 dan satış yapıldı mevcut bakiye  9.131425305201045 ma5 =  48914.213999999956 ma10 =  49529.266000000025  tarih  2021-04-25 21:59:59.999000
51969.24 dan alım yapıldı mevcut bakiye  9.131425305201045 ma5 =  50509.08599999995 ma10 =  49711.65000000002  tarih  2021-04-26 02:59:59.999000
55695.16 dan satış yapıldı mevcut bakiye  9.786100266257908 ma5 =  56226.36999999995 ma10 =  56831.39300000002  tarih  2021-05-04 02:59:59.999000
57636.74 dan alım yapıldı mevcut bakiye  9.786100266257908 ma5 =  56309.74999999994 ma10 =  55724.06500000002  tarih  2021-05-05 15:59:59.9990

# 3 Ema İle Güncel Olarak Al-Sat Yapma